# Stellarium http API with Python and serial controller

* Documentation: https://stellarium.org/doc/head/remoteControlApi.html
* _get_julian_date_ function source: https://stackoverflow.com/questions/31142181/calculating-julian-date-in-python
* Stellarium configuration source: https://medium.com/@lauradurieux/how-to-use-stellarium-http-api-f0942d2c8820

### Stellarium configuration
See atom MD

### Prerequisites

In [88]:
import requests 
from time import sleep
import datetime
import math
import json 

def get_julian_datetime(date):
    """
    Convert a datetime object into julian float.
    Args:
        date: datetime-object of date in question

    Returns: float - Julian calculated datetime.
    Raises: 
        TypeError : Incorrect parameter type
        ValueError: Date out of range of equation
    """

    # Ensure correct format
    if not isinstance(date, datetime.datetime):
        raise TypeError('Invalid type for parameter "date" - expecting datetime')
    elif date.year < 1801 or date.year > 2099:
        raise ValueError('Datetime must be between year 1801 and 2099')

    # Perform the calculation
    julian_datetime = 367 * date.year - int((7 * (date.year + int((date.month + 9) / 12.0))) / 4.0) + int(
        (275 * date.month) / 9.0) + date.day + 1721013.5 + (
                          date.hour + date.minute / 60.0 + date.second / math.pow(60,
                                                                                  2)) / 24.0 - 0.5 * math.copysign(
        1, 100 * date.year + date.month - 190002.5) + 0.5

    return julian_datetime

### Functions

In [364]:
def focus_to_object(_target = "moon", _mode = "center"):
    """
    focus_to_object points to the desired object
    _target: the name of the object to point, for example "moon" or "sun"
    _mode: can be "center", "zoom", "mark"
    """
    # api-endpoint 
    URL = "http://localhost:8090/api/main/focus"

    _target = "moon"
    _mode = "zoom"

    # defining a params dict for the parameters to be sent to the API 
    PARAMS = {"target":_target, "mode": _mode}

    # sending get request and saving the response as response object 
    r = requests.post(url = URL, params = PARAMS) 
    
def move(_x, _y):
    """
    Navigate in the sky depending on
    _x and _y parameters
    """
    # api-endpoint 
    URL = "http://localhost:8090/api/main/move"

    # defining a params dict for the parameters to be sent to the API 
    PARAMS = {"x":_x, "y": _y}

    # sending get request and saving the response as response object 
    r = requests.post(url = URL, params = PARAMS) 
    
def point_to(_az, _alt):
    """
    Points depending on
    _az(imuth) and _alt(itude)
    angles in degrees
    Azimuth from 0 to 359
    Altitude from 0 to 90
    """
    # api-endpoint 
    URL = "http://localhost:8090/api/main/view"
    
    # converting to standard notation
    if _az >= 360 or _az < 0:
        _az = 0
    if _alt > 90 or _alt <-90:
        _alt = 0
        
    if _az >= 0 and _az <=180:
        _az = -_az+180
    elif _az > 180 and _az < 360:
        _az = -_az+540
    
    _az = _az*math.pi/180
    _alt = _alt*math.pi/180

    # defining a params dict for the parameters to be sent to the API
    PARAMS = {"az":_az, "alt": _alt}

    # sending get request and saving the response as response object 
    r = requests.post(url = URL, params = PARAMS) 
     
    
def zoom(_fov):
    """
    Change the FOV
    """
    # api-endpoint 
    URL = "http://localhost:8090/api/main/fov"

    # defining a params dict for the parameters to be sent to the API 
    PARAMS = {"fov":_fov}

    # sending get request and saving the response as response object 
    r = requests.post(url = URL, params = PARAMS) 

def toggle_view(_element):
    """
    Activate or deactivate the element view.
    _element can be "art", "ecliptic", "grid", "ground"
    """
    # api-endpoint 
    URL = "http://localhost:8090/api/stelaction/do"

    if _element == "art":
        _id="actionShow_Constellation_Art"
    elif _element == "ecliptic":
        _id = "actionShow_Ecliptic_Line"
    elif _element == "grid":
        _id="actionShow_Equatorial_Grid"
    elif _element == "ground":
        _id="actionShow_Ground"
    
    # defining a params dict for the parameters to be sent to the API 
    PARAMS = {"id":_id}

    # sending get request and saving the response as response object 
    r = requests.post(url = URL, params = PARAMS) 
    
def time_travel(_mode, _speed = 0.01):
    """
    To speed up or slow down time, or going
    back to present.
    _mode can be "shift" or "now"
    _speed is set to 0.01 as default,can be negative
    """
    # api-endpoint 
    URL = "http://localhost:8090/api/main/time"
    
    if _mode == "shift": 
        _timerate=_speed

        # defining a params dict for the parameters to be sent to the API 
        PARAMS = {"timerate":_timerate}

        # sending get request and saving the response as response object 
        r = requests.post(url = URL, params = PARAMS) 
    
    elif _mode == "now":   
        _JDnow = get_julian_datetime(datetime.datetime.now())
    
        # defining a params dict for the parameters to be sent to the API 
        PARAMS = {"time":_JDnow}

        # sending get request and saving the response as response object 
        r = requests.post(url = URL, params = PARAMS) 
        
def retrieve_az_alt():
    """
    text    
    """
    # api-endpoint 
    URL = "http://localhost:8090/api/main/view"
    
    # defining a params dict for the parameters to be sent to the API 
    PARAMS = {"coord":"altAz", "ref":"auto"}

    # sending get request and saving the response as response object 
    r = requests.get(url = URL, params = PARAMS) 

    res = json.loads(r.text)
    temp = res["altAz"]
    temp = temp.strip("[").strip("]").split(",")

    x = float(temp[0])
    z = float(temp[2])
   
    _alt = math.asin(z)*180/math.pi
    
    _az = math.acos(x/(math.cos(math.asin(z))))*180/math.pi
    
    print(_az, _alt)

In [365]:
retrieve_az_alt()

179.76134245445508 2.8028986316003124


In [311]:
for angle in range(0,360):
    for dec in range(0,10):
        azimuth = angle + dec/10
        point_to(azimuth,0)
        sleep(0.01)

KeyboardInterrupt: 

In [69]:
import serial

OFFSET_AZIMUTH = 0
OFFSET_INCLINATION = 0


ser = serial.Serial('/dev/ttyUSB2', 115200)

while ser.inWaiting()!=0:
    trash = ser.readline()

azimuth_list = []
inclination_list = []

idx = 0
first_loop = True
prev_az = 0
prev_incl = 0

while True:
    
    if ser.inWaiting()!=0:
        try:
            incoming = ser.readline().decode("utf-8")
            incoming = incoming.replace("/","")
            incoming = incoming.replace("x","")
            
            if incoming[0] == "&":
                parsed = str(incoming).split(",")
                azimuth_list.append(float(parsed[1]))
                inclination_list.append(float(parsed[2]))
                idx = idx + 1
                
                if idx == 4:
                    azimuth = sum(azimuth_list) / len(azimuth_list) + OFFSET_AZIMUTH
                    inclination = sum(inclination_list) / len(inclination_list) + OFFSET_INCLINATION
                              
                    print("{},{}".format(azimuth,inclination))  
                    point_to(azimuth,inclination)
                    azimuth_list = []
                    inclination_list = []
                    idx = 0
            
        except:
            pass
    

180.01,0.44
180.01,0.4325
180.01,0.43
180.01,0.4225
180.01,0.41
180.01,0.41
180.01,0.4
180.01,0.39250000000000007
180.01749999999998,0.38
180.02,0.38
180.02,0.37250000000000005
180.025,0.365
180.03,0.36
180.03,0.36
180.03,0.35750000000000004
180.03,0.3475
180.03,0.34
180.03,0.34
180.03,0.34
180.03,0.335
180.0375,0.33
180.04,0.32
180.04,0.32
180.04,0.32
180.04500000000002,0.32
180.05,0.32
180.05,0.31
180.0575,0.31
180.06,0.31
180.06,0.3
180.06,0.2975
180.06,0.295
180.06,0.28500000000000003
180.0625,0.2725
180.07,0.27
180.07,0.27
180.08,0.2725
180.0725,0.28
180.075,0.2725
180.08,0.26
180.08,0.25
180.08,0.255
180.08,0.2675
180.08,0.27
180.08,0.2575
180.08,0.25
180.08,0.25
180.0875,0.25
180.09,0.25
180.09,0.25
180.09,0.2475
180.0975,0.24
180.1,0.23
180.1,0.23
180.1,0.2375
180.1,0.2425
180.1,0.2375
180.10750000000002,0.23
180.11,0.23
180.11,0.23
180.11,0.23
180.11,0.235
180.11,0.23
180.1125,0.23
180.12,0.23
180.12,0.23
180.12,0.23
180.12,0.23
180.12,0.23
180.12,0.2275
180.12,0.23
180.125,0.

118.97,18.035
118.19500000000001,21.262500000000003
117.45500000000001,24.965
116.38,28.9325
115.2,32.6375
114.09,36.214999999999996
113.14500000000001,39.49
112.4575,42.2575
111.525,44.94499999999999
110.36000000000001,47.6925
109.1575,50.6175
107.95,53.7175
106.9825,56.682500000000005
105.805,59.59
104.3725,62.4525
103.29,64.66250000000001
103.45,65.13499999999999
103.7975,64.74499999999999
103.69500000000001,64.51249999999999
103.905,64.25
104.1825,63.83
104.425,63.332499999999996
104.64999999999999,62.872499999999995
105.0125,62.2275
106.035,60.6725
107.625,58.0
110.1475,53.7475
112.6725,48.785
114.49249999999999,44.05499999999999
117.085,38.275
118.3175,33.5675
119.58749999999999,29.355
120.615,25.697499999999998
121.27250000000001,23.205
121.7425,21.634999999999998
122.1875,20.1525
122.7175,18.225
122.95500000000001,16.995
123.3425,16.732499999999998
124.1,16.655
124.97749999999999,16.402499999999996
127.2575,15.6125
130.76500000000001,15.139999999999999
135.7625,15.1650000000000

240.71499999999997,14.825
240.705,14.805
240.7125,14.817499999999999
240.7025,14.867499999999998
240.70499999999998,14.8575
240.69,14.867499999999998
240.69,14.86
240.6975,14.8525
240.705,14.852500000000001
240.695,14.885
240.695,14.822500000000002
240.7025,14.844999999999999
240.7175,14.8775
240.7025,14.8725
240.6925,14.867499999999998
240.7025,14.86
240.6975,14.870000000000001
240.7125,14.895000000000001
240.70749999999998,14.855
240.71,14.8725
240.71249999999998,14.88
240.7,14.899999999999999
240.6875,14.895
240.69,14.885000000000002
240.69,14.905
240.69,14.885000000000002
240.69,14.9025
240.7075,14.9175
240.71249999999998,14.8975
240.69,14.905000000000001
240.67499999999998,14.8975
240.68,14.8775
240.6825,14.907499999999999
240.68249999999998,14.9075
240.6825,14.9025
240.70499999999998,14.915
240.6975,14.885000000000002
240.6675,14.912500000000001
240.67499999999998,14.9275
240.6575,14.92
240.685,14.93
240.6925,14.9375
240.7,14.94
240.6975,14.9275
240.68249999999998,14.945
240.6875

240.74,16.1075
240.71499999999997,16.0675
240.70499999999998,16.03
240.7625,16.045
240.805,16.1075
240.78,16.125
240.75,16.1025
240.735,16.095
240.77999999999997,16.075
240.8075,16.11
240.805,16.1125
240.7825,16.0975
240.78,16.105
240.79500000000002,16.090000000000003
240.8,16.115000000000002
240.8025,16.122500000000002
240.8075,16.122500000000002
240.8,16.1125
240.7925,16.1175
240.82750000000001,16.12
240.8325,16.13
240.83,16.1
240.79749999999999,16.1175
240.7875,16.1175
240.78750000000002,16.1125
240.81,16.1325
240.815,16.14
240.82,16.1575
240.8225,16.1375
240.8125,16.15
240.8125,16.1375
240.8175,16.1525
240.82,16.159999999999997
240.83,16.16
240.82,16.1675
240.82250000000002,16.1725
240.835,16.1575
240.8375,16.2
240.8375,16.175
240.8275,16.14
240.82500000000002,16.1625
240.835,16.1575
240.8375,16.177500000000002
240.85,16.1825
240.85,16.175
240.8375,16.18
240.85,16.1725
240.85750000000002,16.1725
240.85,16.1825
240.85750000000002,16.195
240.85250000000002,16.189999999999998
240.86,1

182.41000000000003,16.995
182.45,17.085
182.46,17.16
182.6425,17.110000000000003
182.78499999999997,17.095
182.8475,17.0375
182.9475,16.825
183.21,16.7325
183.4825,16.635
183.6075,16.5875
183.69250000000002,16.465
183.81000000000003,16.35
183.875,16.2925
183.98,16.192500000000003
184.085,16.1525
184.14249999999998,16.192500000000003
184.155,16.11
184.19500000000002,15.98
184.2875,15.855
184.37,15.7225
184.38,15.692499999999999
184.3675,15.587499999999999
184.3875,15.51
184.42000000000002,15.4875
184.435,15.502500000000001
184.47,15.5825
184.46249999999998,15.717500000000001
184.4775,15.934999999999999
184.54999999999998,15.9525
184.66500000000002,15.875
184.64249999999998,15.53
184.395,15.07
184.41250000000002,14.674999999999999
184.2775,14.64
184.025,14.570000000000002
184.07500000000002,14.68
184.04749999999999,14.889999999999999
184.05,15.06
184.0275,14.9625
183.915,14.94
183.935,14.982500000000002
183.99,14.995000000000001
184.02499999999998,15.025
184.1275,15.1025
184.2075,15.1599

...

185.1575,7.910000000000001
185.1525,7.967499999999999
185.1775,7.965
185.225,7.97
185.2425,7.9475
185.255,7.91
185.265,7.9
185.28,7.9
185.2825,7.9075
185.2875,7.907500000000001
185.29,7.9
185.29,7.9
185.29,7.9
185.29500000000002,7.9075
185.305,7.91
185.31,7.91
185.31,7.91
185.31,7.91
185.31,7.8925
185.31,7.8725000000000005
185.31,7.87
185.31,7.8725
185.31,7.8825
185.31,7.89
185.31,7.89
185.31,7.875
185.3075,7.87
185.3,7.865
185.3,7.86
185.3,7.862500000000001
185.3,7.87
185.3,7.8774999999999995
185.3,7.87
185.3075,7.87
185.31,7.87
185.31,7.8675
185.31,7.86
185.31,7.86
185.31,7.87
185.31,7.87
185.31,7.87
185.31,7.8725
185.31,7.88
185.31,7.8725000000000005
185.31,7.87
185.31,7.867500000000001
185.31,7.87
185.31,7.87
185.3125,7.87
185.32,7.87
185.32,7.87
185.32,7.867500000000001
185.32,7.8675
185.32,7.87
185.32,7.87
185.32,7.87
185.32,7.87
185.32,7.865
185.32,7.86
185.32,7.852500000000001
185.32,7.8549999999999995
185.32,7.86
185.3225,7.86
185.32750000000001,7.86
185.33,7.87
185.33,7.87
18

186.08,8.165
186.0825,8.16
186.09,8.17
186.085,8.175
186.08,8.18
186.08,8.18
186.08,8.1775
186.08,8.172500000000001
186.08,8.1675
186.08,8.16
186.08,8.16
186.08,8.1675
186.08,8.1675
186.08,8.165
186.08,8.17
186.08,8.17
186.08,8.17
186.08,8.17
186.08,8.17
186.08,8.1725
186.08,8.1775
186.08,8.18
186.08,8.1775
186.08,8.17
186.08,8.17
186.08,8.17
186.08,8.165
186.08,8.165
186.08,8.1625
186.08,8.16
186.08,8.165000000000001
186.08,8.17
186.08,8.1625
186.08,8.16
186.08,8.1675
186.08,8.17
186.08,8.17
186.08,8.1625
186.08,8.1675
186.08,8.17
186.08,8.1625
186.08,8.17
186.08,8.175
186.08,8.18
186.08,8.17
186.08,8.17
186.08,8.17
186.08,8.17
186.08,8.17
186.08,8.1725
186.08,8.18
186.08,8.18
186.08,8.1775
186.08,8.1775
186.08,8.1775
186.08,8.175
186.08,8.1775
186.08,8.18
186.07750000000001,8.18
186.075,8.18
186.075,8.1775
186.07,8.175
186.07,8.1775
186.07,8.18
186.07,8.18
186.07,8.18
186.07,8.1825
186.07,8.19
186.07,8.19
186.07,8.19
186.07,8.19
186.07,8.19
186.07,8.1875
186.07,8.18
186.07,8.18499999

185.98,8.14
185.98,8.13
185.98,8.137500000000001
185.98,8.14
185.98,8.14
185.98,8.14
185.98,8.14
185.98,8.14
185.98,8.137500000000001
185.98,8.1425
185.98,8.14
185.98,8.145
185.98,8.147499999999999
185.98,8.15
185.98,8.145
185.98,8.145
185.98,8.15
185.98,8.15
185.98,8.15
185.98,8.15
185.98,8.15
185.98,8.15
185.98,8.15
185.98,8.15
185.98,8.15
185.98,8.15
185.98,8.15
185.98,8.15
185.98,8.15
185.98,8.15
185.98,8.1525
185.975,8.1575
185.9725,8.155000000000001
185.98,8.15
185.97,8.1525
185.97,8.15
185.97,8.145
185.97,8.147499999999999
185.97,8.145
185.97,8.1425
185.97,8.149999999999999
185.97,8.16
185.97,8.16
185.97,8.16
185.9775,8.16
185.9725,8.155
185.97,8.1475
185.97,8.1475
185.975,8.15
185.98,8.1575
185.9725,8.16
185.97,8.1575
185.98,8.15
185.9725,8.145
185.97,8.14
185.97,8.14
185.97,8.1425
185.97,8.15
185.9625,8.145
185.96,8.1425
185.96,8.14
185.96,8.14
185.95499999999998,8.14
185.95,8.1475
185.9475,8.155000000000001
185.945,8.1625
185.95,8.15
185.96,8.14
185.95499999999998,8.1425
185.

185.8,8.137500000000001
185.805,8.13
185.81,8.13
185.81,8.1425
185.8,8.1525
185.81,8.16
185.81,8.1575
185.8025,8.15
185.8,8.137500000000001
185.8,8.137500000000001
185.81,8.14
185.8025,8.15
185.8,8.15
185.8,8.15
185.8,8.1475
185.8,8.14
185.8,8.14
185.8,8.14
185.8,8.14
185.8,8.14
185.8,8.145
185.8,8.1475
185.8,8.15
185.8,8.15
185.8,8.15
185.8,8.15
185.7975,8.15
185.79,8.1575
185.79,8.15
185.79,8.145
185.79,8.147499999999999
185.79,8.15
185.79,8.1475
185.79,8.14
185.79,8.145
185.785,8.14
185.78,8.14
185.78,8.14
185.78,8.14
185.78,8.14
185.78,8.14
185.78,8.14
185.78,8.14
185.78,8.14
185.78,8.14
185.78,8.145
185.78,8.15
185.78,8.15
185.78,8.15
185.78,8.147499999999999
185.78,8.1425
185.78,8.14
185.78,8.1425
185.78,8.1525
185.78,8.1425
185.78,8.145
185.78,8.137500000000001
185.78,8.14
185.78,8.14
185.77,8.14
185.775,8.1425
185.78,8.145
185.78,8.14
185.78,8.14
185.78,8.14
185.78,8.1425
185.7775,8.145
185.77,8.14
185.77,8.14
185.77,8.14
185.77,8.1425
185.77,8.14
185.77,8.14
185.77,8.14
185.77

KeyboardInterrupt: 